In [1]:
%matplotlib inline

import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [2]:
#Read in CSV data files

airbnb = pd.read_csv('zip_clean.csv',index_col=0)
bydz_availcount = pd.read_csv('bydz_availcount.csv',header=None)
bydz_availcount.columns=['date_zip','avail_count']
bydzrt_roomcount = pd.read_csv('bydzrt_roomcount.csv',header=None)
bydzrt_roomcount.columns=['date_zip','room_type','date','bydzrt_count']
prr_df = pd.read_csv('prr_df.csv',index_col=0)
mlp_df = pd.read_csv('mlp_df.csv',index_col=0)
air_zillow = pd.read_csv('airzillow_df.csv', index_col=0)

/Users/Administrator/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
data = pd.read_csv('Gaz_zcta_national.txt', sep="\t")
data.columns = ["zcta", "pop10", "hu10", "aland","awater","aland_sqmi","awater_sqmi","lat","long"]
data.head()


,zcta,pop10,hu10,aland,awater,aland_sqmi,awater_sqmi,lat,long
0,601,18570,7744,166659789,799296,64.348,0.309,18.180555,-66.749961
1,602,41520,18073,79288158,4446273,30.613,1.717,18.362268,-67.176130
2,603,54689,25653,81880442,183425,31.614,0.071,18.455183,-67.119887
3,606,6615,2877,109580061,12487,42.309,0.005,18.158345,-66.932911
4,610,29016,12618,93021467,4172001,35.916,1.611,18.290955,-67.125868


In [3]:
airbnb15 = airbnb.loc[airbnb['year'] == 2015]
airbnb15.to_csv('airbnb15.csv')
airbnb16 = airbnb.loc[airbnb['year'] == 2016]
airbnb16.to_csv('airbnb16.csv')
airbnb17 = airbnb.loc[airbnb['year'] == 2017]
airbnb17.to_csv('airbnb17.csv')

In [55]:
airbnb17_count = airbnb17[['id','date_zip','zipcode','month']].groupby(['date_zip','zipcode','month'], as_index=False).agg('count')
airbnb17_count = pd.merge(airbnb17_count, data[['zcta','pop10','lat','long']],how='left', left_on='zipcode', right_on='zcta')
airbnb17_count = airbnb17_count[['date_zip','zipcode','month','id','pop10','lat','long']]
airbnb17_count = airbnb17_count.rename(columns={'id': 'airbnb_count'})
airbnb17_count['norm_count'] = airbnb17_count['airbnb_count'].div(airbnb17_count['pop10'])
nullpop = airbnb17_count.loc[airbnb17_count['pop10']==0][['date_zip','zipcode','month','airbnb_count','pop10','lat','long']]
nullpop
airbnb17_count = airbnb17_count.loc[airbnb17_count['pop10']!=0]
airbnb17_count
airbnb17_count.to_csv('airbnb17_count.csv')

# print(type(airbnb17_count['zipcode']))
# print(type(data['zcta']))

In [56]:
airbnb17_countavg = airbnb17_count[['zipcode','airbnb_count','pop10','norm_count']].groupby(['zipcode'],as_index=False).agg('mean')
airbnb17_countavg.to_csv('airbnb17_countavg.csv')

In [70]:
sortnorm_17 = airbnb17_countavg.sort_values(by='norm_count',ascending=False)
topnorm_17 = sortnorm_17.loc[sortnorm_17['norm_count']>=0.01]
sortnorm_17

,zipcode,airbnb_count,pop10,norm_count
15,10018,237.500000,5229.0,0.045420
32,10036,766.700000,24711.0,0.031027
10,10012,743.900000,24090.0,0.030880
106,11222,1005.500000,36934.0,0.027224
12,10014,765.900000,31959.0,0.023965
95,11211,2131.400000,90117.0,0.023651
5,10006,70.900000,3011.0,0.023547
121,11238,1144.600000,49262.0,0.023235
16,10019,993.800000,42870.0,0.023182
100,11216,1188.200000,54316.0,0.021876


In [76]:
mlp_avg = mlp_df[['median_list_price','year','zip']].groupby(['zip','year'], as_index=False).agg('mean')
mlp_avg17 = mlp_avg[['median_list_price','year','zip']].loc[mlp_avg['year']==2017]
mlp_avg17

,median_list_price,year,zip
1,3.030000e+06,2017,10001
5,1.588250e+06,2017,10002
8,2.546750e+06,2017,10003
11,1.150167e+06,2017,10004
20,1.439875e+06,2017,10005
23,2.747083e+06,2017,10006
26,4.716831e+06,2017,10007
29,2.424167e+06,2017,10010
34,2.647498e+06,2017,10011
37,4.394583e+06,2017,10012


In [92]:
airbnb17_price = airbnb17[['zipcode','price']].groupby(['zipcode'], as_index=False).agg('mean')

In [93]:
topzip17 = pd.merge(topnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
topzip17 = topzip17[['zipcode','norm_count','price']]
topzip17.to_csv('topzip17.csv')
topzip17

,zipcode,norm_count,price
0,10018,0.045420,238.534737
1,10036,0.031027,212.927612
2,10012,0.030880,224.182417
3,11222,0.027224,122.862755
4,10014,0.023965,241.244027
5,11211,0.023651,145.238951
6,10006,0.023547,196.968970
7,11238,0.023235,135.176830
8,10019,0.023182,213.625377
9,11216,0.021876,107.288924


In [94]:
topnormprice_17 = topzip17.sort_values(by='price',ascending=False)
topnormprice_17

,zipcode,norm_count,price
26,10017,0.013599,263.463177
19,10013,0.016264,244.102331
4,10014,0.023965,241.244027
0,10018,0.045420,238.534737
10,10001,0.021173,227.335049
18,10011,0.016996,225.136757
2,10012,0.030880,224.182417
32,10010,0.010555,222.404464
13,10003,0.019202,219.645752
8,10019,0.023182,213.625377


In [96]:
sortnorm_17

norm_17 = pd.merge(sortnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
norm_17 = norm_17[['zipcode','norm_count','price']]
norm_17.to_csv('norm17.csv')
norm_17

,zipcode,norm_count,price
0,10018,0.045420,238.534737
1,10036,0.031027,212.927612
2,10012,0.030880,224.182417
3,11222,0.027224,122.862755
4,10014,0.023965,241.244027
5,11211,0.023651,145.238951
6,10006,0.023547,196.968970
7,11238,0.023235,135.176830
8,10019,0.023182,213.625377
9,11216,0.021876,107.288924
